In [1]:
import os as os
import csv as csv
import requests as req

# What is this?
# Download SRA datasets from EBI
# Can run for quite some time...

source_file = '/home/pebert/work/code/mpggit/crossspecies/annotation/datasrc/sra/20170818_SRA_expression_fastq.tsv'

fastq_dest = '/TL/deep/fhgfs/projects/pebert/thesis/projects/cross_species/rawdata/temp'

ena_check = 'http'
ena_load = 'ftp'

ena_fastq = '{protocol}://ftp.sra.ebi.ac.uk/vol1/fastq/{run_short}/{run_full}/{read}'

def check_available(url):
    resp = req.head(url)
    code, length = resp.status_code, int(resp.headers['Content-Length']) // (1024 * 1024)
    return code, length

def load_existing_samples(path, dtype):
    files = os.listdir(path)
    exists = [f for f in files if f.startswith('SRR')]
    if dtype == 'sra':
        runid = [f.split('.')[0] for f in exists]
    else:
        exists = [f for f in files if f.startswith('SRR')]
        runid = [f.split('.')[0] for f in exists]
        runid = set(runid).union([f.split('_')[0] for f in exists])
    return set(runid).union(set(exists))

exist_fastq = load_existing_samples(fastq_dest, 'fastq')

good, bad = [], []
with open(source_file, 'r', newline='') as source:
    rows = csv.DictReader(source, delimiter='\t')
    for row in rows:
        runid = row['sraid']
        try:
            use = bool(int(row['use']))
        except ValueError:
            continue
        if runid and use:
            short = runid[:6]
            read_1 = '{}_1.fastq.gz'.format(runid)
            read_2 = '{}_2.fastq.gz'.format(runid)
            to_load = []
            if read_1 not in exist_fastq:
                to_load.append(read_1)
            if read_2 not in exist_fastq:
                to_load.append(read_2)
            for item in to_load:
                chk_url = ena_fastq.format(**{'run_short': short, 'run_full': runid,
                                              'read': item, 'protocol': ena_check})
                load_url = ena_fastq.format(**{'run_short': short, 'run_full': runid,
                                               'read': item, 'protocol': ena_load})
                c, l = check_available(chk_url)
                if c == 200:
                    good.append(load_url)
                else:
                    bad.append(load_url)
                
with open(os.path.join(fastq_dest, 'ena_load.txt'), 'w') as dump:
    good.append('')
    _ = dump.write('\n'.join(good))
    
with open('/home/pebert/ena_bad_urls.txt', 'w') as dump:
    bad.append('')
    _ = dump.write('\n'.join(bad))